### Building a chatbot with multiple tools using LangGraph 
### Aim : To create a chatbot with multiple tools that includes Arxiv Search , Wikipidea Search and other functions

In [4]:
from dotenv import load_dotenv

from langchain_community.tools import ArxivQueryRun , WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper , WikipediaAPIWrapper
from langchain_community.tools.tavily_search import TavilySearchResults

from langchain_groq import ChatGroq

In [5]:
load_dotenv()

True

In [6]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results= 2 , doc_content_chars_max= 500) 
arxiv = ArxivQueryRun(api_wrapper= arxiv_wrapper)

In [10]:
arxiv.invoke("Attention is all you need") 

'Published: 2021-05-06\nTitle: Do You Even Need Attention? A Stack of Feed-Forward Layers Does Surprisingly Well on ImageNet\nAuthors: Luke Melas-Kyriazi\nSummary: The strong performance of vision transformers on image classification and other vision tasks is often attributed to the design of their multi-head attention layers. However, the extent to which attention is responsible for this strong performance remains unclear. In this short report, we ask: is the attention layer even necessary? Specifi'

In [13]:
#pip install wikipedia

In [7]:
wiki_wrapper = WikipediaAPIWrapper(top_k_results= 1 , doc_content_chars_max=500)
wiki = WikipediaQueryRun(api_wrapper= wiki_wrapper)

In [15]:
wiki.invoke("Pokemon")

"Page: Pokémon\nSummary: Pokémon is a Japanese media franchise consisting of video games, animated series and films, a trading card game, and other related media. The franchise takes place in a shared universe in which humans co-exist with creatures known as Pokémon, a large variety of species endowed with special powers. The franchise's primary target audience is children aged 5 to 12, but it is known to attract people of all ages. Pokémon is estimated to be the world's highest-grossing media fra"

In [8]:
tavily = TavilySearchResults()

/var/folders/9c/tpzdhn2d0rv1nscqt75gbvgw0000gn/T/ipykernel_32659/2290291758.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  tavily = TavilySearchResults()


In [9]:
tavily.invoke("Recent AI news")

[{'title': 'The latest AI news we announced in October - Google Blog',
  'url': 'https://blog.google/technology/ai/google-ai-updates-october-2025/',
  'content': '"The Latest AI News We Announced in October" highlights Google\'s recent AI advancements.\n Google\'s AI breakthroughs include a quantum algorithm faster than supercomputers.\n An AI model called Cell2Sentence-Scale may offer a new cancer therapy pathway.\n Gemini Enterprise is now the "front door" for Google AI in the workplace.\n New AI security features protect against scams, plus Google Home gets AI upgrades.\n\n## Shakespeare-ish [...] # The latest AI news we announced in October\n\nNov 04, 2025\n\nHere’s a recap of some of our biggest AI updates from October, including Gemini Enterprise, an AI model to accelerate cancer research and a big quantum computing breakthrough.\n\nSuperG\n\n## General summary [...] For more than 20 years, we’ve invested in machine learning and AI research, tools and infrastructure to build prod

In [20]:
tools = [arxiv , wiki , tavily]

In [22]:
llm = ChatGroq( model = "openai/gpt-oss-20b"   
)

In [23]:
llm_tool = llm.bind_tools(tools)

In [28]:
response = llm_tool.invoke("Arxiv : Attention is all you need")
print(response)
print(response.tool_calls)

content='' additional_kwargs={'reasoning_content': 'We need to use the arxiv function to search for the paper "Attention is all you need".', 'tool_calls': [{'id': 'fc_06c25e3d-fce8-41b8-bfd8-20591891b792', 'function': {'arguments': '{"query":"Attention is all you need"}', 'name': 'arxiv'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 293, 'total_tokens': 341, 'completion_time': 0.047967461, 'prompt_time': 0.015940768, 'queue_time': 0.003340198, 'total_time': 0.063908229, 'completion_tokens_details': {'reasoning_tokens': 21}}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_d08f3e4cc0', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--a2d7c945-f55a-443d-bd53-a3b26e81d959-0' tool_calls=[{'name': 'arxiv', 'args': {'query': 'Attention is all you need'}, 'id': 'fc_06c25e3d-fce8-41b8-bfd8-20591891b792', 'type': 'tool_call'}] usage_metadata={'input_tokens': 29

# Chatbot with Langgraph

In [10]:
from langgraph.graph import StateGraph , START , END
from langgraph.prebuilt import ToolNode , tools_condition

from typing_extensions import TypedDict
from langgraph.graph.message import add_messages , Annotated 
from langchain.messages import HumanMessage , AIMessage , AnyMessage

In [37]:
class State(TypedDict) :
    messages : Annotated[list[AnyMessage] , add_messages]

In [44]:
graph = StateGraph(State)

In [45]:
tools = [arxiv , wiki , tavily]

In [46]:
llm = ChatGroq( model = "openai/gpt-oss-20b"   
)

llm_tool = llm.bind_tools(tools)

In [47]:
# Node Definitions 

def chatbot(state : State) :

    return {"messages" : [llm_tool.invoke(state["messages"])]}

In [48]:
# Create Nodes 
graph.add_node("chatbot" , chatbot)
graph.add_node("tools", ToolNode(tools) )


graph.add_edge(START, "chatbot")
graph.add_conditional_edges("chatbot" , tools_condition)
graph.add_edge("tools" , END)

graph_builder = graph.compile()

In [50]:
graph_builder.invoke({"messages" :["Recent AI News"] })


{'messages': [HumanMessage(content='Recent AI News', additional_kwargs={}, response_metadata={}, id='cc702d61-efde-4f25-bb51-fb9b37bba761'),
  AIMessage(content='', additional_kwargs={'reasoning_content': 'User says "Recent AI News". Likely wants recent news about AI. We should browse. Use tavily_search_results_json.', 'tool_calls': [{'id': 'fc_e21a47d8-c784-4851-af46-20a326ea79d2', 'function': {'arguments': '{"query":"Recent AI news"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 288, 'total_tokens': 344, 'completion_time': 0.055990089, 'prompt_time': 0.015308664, 'queue_time': 0.003248917, 'total_time': 0.071298753, 'completion_tokens_details': {'reasoning_tokens': 27}}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_e0f4f3edeb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--2d5dd7f7-7036-4007-8287-d850b9b